In [12]:
import torch
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, LlamaForCausalLM
import os
import json
import random
from typing import *
import pandas as pd
from statsmodels.stats.proportion import proportion_confint

In [13]:
def load_gpt_model_and_tokenizer(model_name:str, device='cuda'):
    """
    Loads a huggingface model and its tokenizer

    Parameters:
    model_name: huggingface name of the model to load (e.g. GPTJ: "EleutherAI/gpt-j-6B", or "EleutherAI/gpt-j-6b")
    device: 'cuda' or 'cpu'

    Returns:
    model: huggingface model
    tokenizer: huggingface tokenizer
    MODEL_CONFIG: config variables w/ standardized names

    """
    assert model_name is not None

    print("Loading: ", model_name)

    if 'gpt-j' in model_name.lower():
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        tokenizer.pad_token = tokenizer.eos_token
        model = AutoModelForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True).to(device)

        MODEL_CONFIG={"n_heads":model.config.n_head,
                      "n_layers":model.config.n_layer,
                      "resid_dim":model.config.n_embd,
                      "name_or_path":model.config.name_or_path,
                      "attn_hook_names":[f'transformer.h.{layer}.attn.out_proj' for layer in range(model.config.n_layer)],
                      "layer_hook_names":[f'transformer.h.{layer}' for layer in range(model.config.n_layer)],
                      "prepend_bos":False}

    elif 'gpt2' in model_name.lower():
        tokenizer = AutoTokenizer.from_pretrained("gpt2")
        tokenizer.pad_token = tokenizer.eos_token
        model = AutoModelForCausalLM.from_pretrained("gpt2")

        MODEL_CONFIG={"n_heads":model.config.n_head,
                      "n_layers":model.config.n_layer,
                      "resid_dim":model.config.n_embd,
                      "name_or_path":model.config.name_or_path,
                      "attn_hook_names":[f'transformer.h.{layer}.attn.out_proj' for layer in range(model.config.n_layer)],
                      "layer_hook_names":[f'transformer.h.{layer}' for layer in range(model.config.n_layer)],
                      "prepend_bos":False}

    elif 'gpt-neo-125m' in model_name.lower():
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        tokenizer.pad_token = tokenizer.eos_token
        model = AutoModelForCausalLM.from_pretrained(model_name)

        MODEL_CONFIG={"n_heads":model.config.num_heads,
                      "n_layers":model.config.num_layers,
                      "resid_dim": model.config.hidden_size,
                      "name_or_path":model.config.name_or_path,
                      "attn_hook_names":[f'gpt_neo.layers.{layer}.attention.dense' for layer in range(model.config.num_layers)],
                      "layer_hook_names":[f'gpt_neo.layers.{layer}' for layer in range(model.config.num_layers)],
                      "prepend_bos":False}

    elif 'gpt-neox' in model_name.lower() or 'pythia' in model_name.lower():
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        tokenizer.pad_token = tokenizer.eos_token
        model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)

        MODEL_CONFIG={"n_heads":model.config.num_attention_heads,
                      "n_layers":model.config.num_hidden_layers,
                      "resid_dim": model.config.hidden_size,
                      "name_or_path":model.config.name_or_path,
                      "attn_hook_names":[f'gpt_neox.layers.{layer}.attention.dense' for layer in range(model.config.num_hidden_layers)],
                      "layer_hook_names":[f'gpt_neox.layers.{layer}' for layer in range(model.config.num_hidden_layers)],
                      "prepend_bos":False}

    elif 'llama' in model_name.lower():
        if '70b' in model_name.lower():
            # use quantization. requires `bitsandbytes` library
            from transformers import BitsAndBytesConfig
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type='nf4',
                bnb_4bit_use_double_quant=True,
                bnb_4bit_compute_dtype=torch.float16
            )

            access_token = "hf_findNewOne"

            tokenizer = LlamaTokenizer.from_pretrained(model_name, token=access_token)
            model = LlamaForCausalLM.from_pretrained(
                model_name,
                trust_remote_code=True,
                quantization_config=bnb_config,
                token=access_token
            )
        else:
            if '7b' in model_name.lower() or '8b' in model_name.lower():
                model_dtype = torch.float32
            else: #half precision for bigger llama models
                #This becomes only for the 13B model then. Okay then. What else?
                model_dtype = torch.float16

            # If transformers version is < 4.31 use LlamaLoaders
            # tokenizer = LlamaTokenizer.from_pretrained(model_name)
            # model = LlamaForCausalLM.from_pretrained(model_name, torch_dtype=model_dtype).to(device)

            # If transformers version is >= 4.31, use AutoLoaders
            access_token = "hf_findNewOne"

            tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)
            model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=model_dtype, token=access_token).to(device)

        MODEL_CONFIG={"n_heads":model.config.num_attention_heads,
                      "n_layers":model.config.num_hidden_layers,
                      "resid_dim":model.config.hidden_size,
                      "name_or_path":model.config._name_or_path,
                      "attn_hook_names":[f'model.layers.{layer}.self_attn.o_proj' for layer in range(model.config.num_hidden_layers)],
                      "layer_hook_names":[f'model.layers.{layer}' for layer in range(model.config.num_hidden_layers)],
                      "prepend_bos":True}
    else:
        raise NotImplementedError("Still working to get this model available!")


    return model, tokenizer, MODEL_CONFIG

In [14]:
model_technical_name = "EleutherAI/gpt-neo-125m"
#model_technical_name = 'gpt-2'

model, tokenizer, model_config = load_gpt_model_and_tokenizer(model_technical_name)

Loading:  EleutherAI/gpt-neo-125m


In [15]:
tokenizer.vocab['Asia']

38555

In [19]:
inputs = tokenizer('Asia', return_tensors="pt")

In [24]:
embedding_layer = model.transformer.wte
static_embedding = embedding_layer(inputs["input_ids"])

In [25]:
static_embedding

tensor([[[ 4.9023e-01, -8.6328e-01,  1.8555e-01, -1.6406e-01, -2.8125e-01,
           1.1182e-01,  4.5312e-01, -2.2070e-01,  5.2734e-01,  5.2344e-01,
           3.6011e-03,  5.7422e-01,  3.7695e-01,  8.3594e-01, -4.6387e-02,
          -1.3867e-01,  2.1289e-01, -2.9492e-01,  7.1289e-02, -6.6016e-01,
           5.6250e-01,  3.5352e-01, -1.2354e-01,  4.7656e-01,  3.7109e-01,
           7.4707e-02, -2.1582e-01, -1.6113e-01,  6.0156e-01, -1.5332e-01,
          -1.8555e-02,  2.7930e-01,  1.7871e-01, -5.1562e-01, -1.0498e-01,
           3.8086e-02,  1.2158e-01,  1.8457e-01, -1.7212e-02,  9.6875e-01,
           2.0996e-01, -4.7461e-01, -4.3457e-02,  3.5547e-01,  7.1777e-02,
           6.6895e-02, -1.0156e-01,  3.3398e-01, -4.8047e-01,  7.0312e-02,
          -3.4180e-01, -1.8652e-01, -1.0205e-01,  2.0312e-01,  3.4961e-01,
           5.4443e-02, -4.6484e-01, -1.5723e-01,  2.1582e-01,  2.9541e-02,
          -5.5469e-01, -3.0273e-01, -5.4932e-02,  3.5742e-01,  1.5527e-01,
          -3.6133e-01,  4

In [26]:
embedding_matrix = model.transformer.wte.weight

In [27]:
embedding_matrix

Parameter containing:
tensor([[ 0.1709, -0.7383,  0.4277,  ...,  0.0840,  0.5820, -0.3457],
        [ 0.2070, -0.6055,  0.4590,  ...,  0.1562,  0.4883, -0.2363],
        [ 0.2324, -0.6367,  0.3262,  ...,  0.2236,  0.7500, -0.2354],
        ...,
        [ 0.7734, -1.1406,  0.6523,  ...,  0.2832,  0.9258, -0.5547],
        [ 0.3906, -0.8438,  0.5117,  ...,  0.0148,  0.6992, -0.2383],
        [ 0.2734, -0.7148,  0.2949,  ...,  0.1748,  0.4043, -0.3105]],
       requires_grad=True)

In [29]:
subsample = torch.randperm(model.config.vocab_size)[:5000].to(model.device)

In [30]:
embedding_layer = model.transformer.wte
subsampled_embeddings = embedding_layer(subsample)

In [31]:
W_E_normed = subsampled_embeddings / subsampled_embeddings.norm(dim=-1, keepdim=True)

In [32]:
cosine_sims = W_E_normed @ W_E_normed.T


In [33]:
cosine_sims

tensor([[1.0000, 0.9306, 0.8318,  ..., 0.8345, 0.8597, 0.8456],
        [0.9306, 1.0000, 0.8563,  ..., 0.8721, 0.8763, 0.8769],
        [0.8318, 0.8563, 1.0000,  ..., 0.9268, 0.9267, 0.9263],
        ...,
        [0.8345, 0.8721, 0.9268,  ..., 1.0000, 0.9187, 0.9150],
        [0.8597, 0.8763, 0.9267,  ..., 0.9187, 1.0000, 0.9168],
        [0.8456, 0.8769, 0.9263,  ..., 0.9150, 0.9168, 1.0000]],
       grad_fn=<MmBackward0>)